# Joining molecule data

## Mapping molecules to pdb structures

**Data source**:

* Open Targets molecule index: `gs://ot-snapshots/etl/outputs/21.04.2/parquet/molecule`
* PDBe Chem dataset used to [Chemical Component Dictionary identifier](https://www.wwpdb.org/data/ccd) to [inchiKeys](https://en.wikipedia.org/wiki/International_Chemical_Identifier#InChIKey): `ftp://ftp.ebi.ac.uk/pub/databases/msd/pdbechem_v2/components_inchikeys.csv`
* ePDB-KB REST API for retrieving pdb structures with the given compound (CCD) identifiers in complex.

## Mapping chains in pdb structures to uniprot/ensembl

**data source**:

* SIFTS dataset downloaded from EBI ftp: `ftp://ftp.ebi.ac.uk/pub/databases/msd/sifts/flatfiles/csv/pdb_chain_ensembl.csv.gz`

In [6]:
%%bash

# Create folders for data:
mkdir -p data
rm -rf data/*

# Fetch molecule index:
gsutil cp -r gs://ot-snapshots/etl/outputs/21.04.2/parquet/molecule data/ 2> /dev/null

# Fetch target index:
gsutil cp -r gs://ot-snapshots/etl/outputs/21.04.2/parquet/targets data/ 2> /dev/null

# Fetch pdbeChem mapping file:
wget -q ftp://ftp.ebi.ac.uk/pub/databases/msd/pdbechem_v2/components_inchikeys.csv -P data/

# Fetch SIFTS mapping file:
wget -q ftp://ftp.ebi.ac.uk/pub/databases/msd/sifts/flatfiles/csv/pdb_chain_ensembl.csv.gz -P data/
ls -lah data/

total 20792
drwxr-xr-x   6 dsuveges  384566875   192B 22 Apr 00:30 .
drwxrwxr-x   6 dsuveges  384566875   192B 22 Apr 00:27 ..
-rw-r--r--   1 dsuveges  384566875   1.2M 22 Apr 00:30 components_inchikeys.csv
drwxr-xr-x  70 dsuveges  384566875   2.2K 22 Apr 00:30 molecule
-rw-r--r--   1 dsuveges  384566875   8.6M 22 Apr 00:30 pdb_chain_ensembl.csv.gz
drwxr-xr-x  29 dsuveges  384566875   928B 22 Apr 00:30 targets


### Stats on the input files:

In [10]:
import pandas as pd
import json
import requests

from pyspark.sql.types import ArrayType, StringType, IntegerType
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

# establish spark connection
spark = (
    SparkSession.builder
    .master('local[*]')
    .getOrCreate()
)


#### Molecule index/CCD index and the joined dataset

In [13]:
molecules = (
    spark.read.parquet('data/molecule/')
    .select(
        f.col('id').alias('chemblId'),
        f.col('inchiKey'),
        f.col('name')
    )
    .distinct()
    .persist()
)

# Get some basic stats:
print(f'Number of molecules: {molecules.count()}')
print(f'Number of molecules without inchiKeys: {molecules.filter(f.col("inchiKey").isNull()).count()}')

Number of molecules: 13076
Number of molecules without inchiKeys: 2268


In [25]:
inchikey_mapping = (
    spark.read.csv('data/components_inchikeys.csv', sep=',', header=True)
    .withColumnRenamed('inchiKey', 'InChIKey')
    .withColumnRenamed('CCD_ID', 'compoundId')
    .persist()
)

# Get some basic stats:
print(f'Number of molecules: {inchikey_mapping.count()}')
print(f'Number of molecules without inchiKeys: {inchikey_mapping.filter(f.col("inchiKey").isNull()).count()}')


Number of molecules: 36905
Number of molecules without inchiKeys: 137


In [35]:
# There are duplications:
ambiguity = (
    inchikey_mapping
    .filter(f.col('inchiKey').isNotNull())
    .groupBy('inchiKey')
    .agg(
        f.collect_set(f.col('compoundId')).alias('compoundIds')
    )
    .filter(f.size(f.col('compoundIds')) > 1)
    .orderBy(f.size(f.col('compoundIds')), ascending=False)
    .persist()
)

print(f'Number of inchy keys with multiple compounds: {ambiguity.count()}')
ambiguity.show(truncate=False)

Number of inchy keys with multiple compounds: 221
+---------------------------+-----------------------------+
|inchiKey                   |compoundIds                  |
+---------------------------+-----------------------------+
|XLYOFNOQVPJJNP-UHFFFAOYAF  |[DIS, OX, GTE, MTO, QTR, OXO]|
|DJZPHYIXNUOVJU-VYUIOLGVSA-N|[50M, 53M, 53L, 54Q]         |
|RWSXRVCMGQZWBV-WDSKDSINSA-O|[GTT, VDW, 0P0]              |
|SNDPXSYFESPGGJ-XWEZEGGSDX  |[2PI, BTA, RON]              |
|IIXWYSCJSQVBQM-LLVKDONJSA-N|[QB4, 53P, 5P8]              |
|QGZKDVFQNNGYKY-UHFFFAOYAF  |[NGN, NH, NH2]               |
|LTFMZDNNPPEQNG-KVQBGUIXSA-N|[DGP, PG7, DG]               |
|LFQSCWFLJHTTHZ-UHFFFAOYAB  |[EOX, OHE, OXA]              |
|NLTUCYMLOPLUHL-KQYNXXCUSA-N|[SAP, AGS, ATG]              |
|VOKZMFPBFFRNPZ-IVZWLZJFSA-N|[4PD, 4PE, 4PC]              |
|FHBXKBNKQMSUIJ-SHYZEUOFSA-N|[C7R, C7S, SC]               |
|RQFCJASXJCIDSX-UUOKFMHZSA-N|[G25, G, 5GP]                |
|HAEJPQIATWHALX-KQYNXXCUSA-N|[CZU, ITT]           

In [37]:
# There are duplications:
ambiguity = (
    inchikey_mapping
    .filter(f.col('compoundId').isNotNull())
    .groupBy('compoundId')
    .agg(
        f.collect_set(f.col('inchiKey')).alias('inchiKey')
    )
    .filter(f.size(f.col('inchiKey')) > 1)
    .orderBy(f.size(f.col('inchiKey')), ascending=False)
    .persist()
)

print(f'Number of compound ids with multiple inchi keys: {ambiguity.count()}')
ambiguity.show(truncate=False)

Number of compound ids with multiple inchi keys: 0
+----------+--------+
|compoundId|inchiKey|
+----------+--------+
+----------+--------+



In [39]:
molecule_w_compounds = (
    molecules
    .join(inchikey_mapping, on='inchiKey', how='inner')
    .drop('inchiKey')
    .persist()
)

# Get some basic stats:
print(f'Number of overlapping molecules: {molecule_w_compounds.count()}')
print(f'Ambiguity in the joining: {molecule_w_compounds.select(f.col("chemblId")).distinct().count()}')


Number of overlapping molecules: 3727
Ambiguity in the joining: 3671


In [42]:
ambiguity = (
    molecule_w_compounds
    .groupBy('chemblId', 'name')
    .agg(
       f.collect_set(f.col('compoundId')).alias('compoundId')
    )
    .filter(f.size(f.col('compoundId')) > 1)
    .orderBy(f.size(f.col('compoundId')), ascending=False)
)

print(f'Number of molecules with different compound ids: {ambiguity.count()}')
ambiguity.show(truncate=False)

Number of molecules with different compound ids: 52
+-------------+-------------------------------------------+---------------+
|chemblId     |name                                       |compoundId     |
+-------------+-------------------------------------------+---------------+
|CHEMBL283807 |CHEMBL283807                               |[G25, G, 5GP]  |
|CHEMBL131890 |ATPGAMMAS                                  |[SAP, AGS, ATG]|
|CHEMBL3286830|LORLATINIB                                 |[QB4, 53P, 5P8]|
|CHEMBL477487 |Deoxyguanosine 5'-Monophosphate            |[DGP, PG7, DG] |
|CHEMBL1437   |NOREPINEPHRINE                             |[LT4, LNR]     |
|CHEMBL58832  |ASPARAGINE                                 |[ASN, 41Q]     |
|CHEMBL1231683|CHEMBL1231683                              |[CCK, ATW]     |
|CHEMBL553426 |(R)-2-Aminobutanoic Acid                   |[DBB, AA3]     |
|CHEMBL1160508|L-Cysteinesulfinic Acid                    |[CSW, CSD]     |
|CHEMBL1160559|QUINALDIC ACID       